In [11]:
import polars as pl

## API Call Sequence

In [12]:
df_dynamic = pl.read_csv("./datasets/api_call_sequences.csv.gz")

print(f"Successfully loaded. Shape: {df_dynamic.shape}")
print("Columns:", df_dynamic.columns)

Successfully loaded. Shape: (43876, 102)
Columns: ['hash', 't_0', 't_1', 't_2', 't_3', 't_4', 't_5', 't_6', 't_7', 't_8', 't_9', 't_10', 't_11', 't_12', 't_13', 't_14', 't_15', 't_16', 't_17', 't_18', 't_19', 't_20', 't_21', 't_22', 't_23', 't_24', 't_25', 't_26', 't_27', 't_28', 't_29', 't_30', 't_31', 't_32', 't_33', 't_34', 't_35', 't_36', 't_37', 't_38', 't_39', 't_40', 't_41', 't_42', 't_43', 't_44', 't_45', 't_46', 't_47', 't_48', 't_49', 't_50', 't_51', 't_52', 't_53', 't_54', 't_55', 't_56', 't_57', 't_58', 't_59', 't_60', 't_61', 't_62', 't_63', 't_64', 't_65', 't_66', 't_67', 't_68', 't_69', 't_70', 't_71', 't_72', 't_73', 't_74', 't_75', 't_76', 't_77', 't_78', 't_79', 't_80', 't_81', 't_82', 't_83', 't_84', 't_85', 't_86', 't_87', 't_88', 't_89', 't_90', 't_91', 't_92', 't_93', 't_94', 't_95', 't_96', 't_97', 't_98', 't_99', 'malware']


In [13]:
display(df_dynamic.head())

hash,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,t_10,t_11,t_12,t_13,t_14,t_15,t_16,t_17,t_18,t_19,t_20,t_21,t_22,t_23,t_24,t_25,t_26,t_27,t_28,t_29,t_30,t_31,t_32,t_33,t_34,t_35,…,t_64,t_65,t_66,t_67,t_68,t_69,t_70,t_71,t_72,t_73,t_74,t_75,t_76,t_77,t_78,t_79,t_80,t_81,t_82,t_83,t_84,t_85,t_86,t_87,t_88,t_89,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,malware
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""071e8c3f8922e186e57548cd4c703a…",112,274,158,215,274,158,215,298,76,208,76,172,117,172,117,172,76,117,35,60,81,60,81,60,81,60,81,60,81,60,81,60,81,60,81,60,…,225,35,60,81,35,225,172,60,81,60,81,60,81,172,117,76,172,117,172,117,35,111,81,140,208,240,117,71,297,135,171,215,35,208,56,71,1
"""33f8e6d08a6aae939f25a8e0d63dd5…",82,208,187,208,172,117,172,117,172,117,172,117,172,117,172,117,172,117,172,117,172,117,172,117,172,117,208,172,117,16,208,297,8,199,264,274,…,208,240,117,35,60,81,60,81,172,35,60,81,60,81,31,60,81,208,187,208,274,158,266,208,60,81,60,81,240,117,71,297,135,171,215,35,1
"""b68abd064e975e1c6d5f25e7486630…",16,110,240,117,240,117,240,117,240,117,240,117,240,117,172,117,99,260,141,65,240,117,240,117,240,117,263,215,263,215,263,215,263,215,263,215,…,65,112,123,65,113,112,123,65,112,123,65,112,123,65,113,112,123,65,112,123,65,112,123,65,113,112,123,65,112,123,65,112,123,65,113,112,1
"""72049be7bd30ea61297ea624ae1980…",82,208,187,208,172,117,172,117,172,117,172,117,172,117,172,117,172,117,172,117,172,117,172,117,172,117,208,16,240,117,228,208,187,228,240,117,…,82,240,117,240,117,240,117,240,117,228,187,215,274,158,215,274,158,215,240,117,71,297,135,171,215,228,215,208,302,208,302,187,208,302,228,302,1
"""c9b3700a77facf29172f32df6bc77f…",82,240,117,240,117,240,117,240,117,172,117,172,117,16,240,117,11,274,158,215,274,158,215,117,270,117,301,117,297,8,199,264,215,260,141,65,…,65,31,159,224,82,261,172,117,260,208,260,2,140,81,208,159,224,82,159,224,82,261,208,240,117,260,40,209,260,40,209,260,141,260,141,260,1


## UCI Hybrid Dataset

In [14]:
df_benign = pl.read_csv("./datasets/staDynBenignLab.csv.gz")
df_vxheaven = pl.read_csv("./datasets/staDynVxHeaven2698Lab.csv.gz")
df_vt = pl.read_csv("./datasets/staDynVt2955Lab.csv.gz")

print(f"Loaded Benign: {df_benign.shape}")
print(f"Loaded VxHeaven: {df_vxheaven.shape}")
print(f"Loaded VirusTotal: {df_vt.shape}")

# Add label 0 (Benign)
df_benign = df_benign.with_columns(pl.lit(0).alias("label"))

# Add label 1 (Malware) - Checks if label col exists, if not adds it, if yes ensures it's 1
# Note: Usually the malware datasets have the label, but we force consistency.
if "Label" not in df_vxheaven.columns:
        df_vxheaven = df_vxheaven.with_columns(pl.lit(1).alias("label"))

if "Label" not in df_vt.columns:
        df_vt = df_vt.with_columns(pl.lit(1).alias("label"))

# Stack the datasets (Vertical Concatenation)
# Since these are different SAMPLES (rows), we stack them.
# We use 'diagonal' to handle any slight column name mismatches safely.
df_hybrid = pl.concat([df_benign, df_vxheaven, df_vt], how="diagonal")

# Cleanup: Drop columns with ANY null values
# The 'diagonal' concat might introduce nulls if columns didn't match perfectly.
# We want only the features present in all datasets.
original_cols = len(df_hybrid.columns)

# Identify columns with nulls
null_cols = [col for col in df_hybrid.columns if df_hybrid[col].null_count() > 0]

# Drop them
df_hybrid = df_hybrid.drop(null_cols)

print(f"Successfully combined. Final Shape: {df_hybrid.shape}")
display(df_hybrid.head())

Loaded Benign: (595, 1087)
Loaded VxHeaven: (2698, 1087)
Loaded VirusTotal: (2955, 1088)
Successfully combined. Final Shape: (6248, 1085)


Virtual,Offset,loc,Import,Imports,var,Forwarder,UINT,LONG,BOOL,WORD,BYTES,large,short,dd,db,dw,XREF,ptr,DATA,FUNCTION,extrn,byte,word,dword,char,DWORD,stdcall,arg,locret,asc,align,WinMain,unk,cookie,off,nullsub,…,file_alignment,major_operating_system_version,minor_operating_system_version,major_image_version,minor_image_version,major_subsystem_version,minor_subsystem_version,size_of_headers,subsystem,dll_characteristics,loader_flags,number_of_imports.1,AddressOfEntryPoint,SizeOfHeaders.1,CheckSum,size_of_stack_reserve,size_of_stack_commit,size_of_heap_reserve,size_of_heap_commit,image_base,Size_image,BaseOfCode,number_of_rva_and_sizes.1,number_of_IAT_entires.1,count_mutex,files_operations,count_file_read,count_file_written,count_file_exists,count_file_deleted,count_file_copied,count_file_renamed,count_regkey_written,count_regkey_deleted,count_file_opened,count_dll_loaded,label
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i32
0,0,1,0,0,0,0,0,0,0,0,0,0,0,421,52,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,512,6,2,6,2,6,0,1024,2,33088,0,5,12810,1024,55744,262144.0,8192.0,1.048576e6,4096.0,4.194304e6,36864,4096,16,85.0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,120,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,512,6,2,6,2,6,2,1024,3,33088,0,4,10505,1024,62768,262144.0,8192.0,1.048576e6,4096.0,4.194304e6,28672,4096,16,94.0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,153,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,512,5,1,5,1,4,0,1024,3,32768,0,5,5442,1024,22559,262144.0,4096.0,1.048576e6,4096.0,1.6777216e7,20480,4096,16,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,765,62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,512,6,1,6,1,6,1,1024,3,33088,0,7,19525,1024,72145,262144.0,8192.0,1.048576e6,4096.0,1.6777216e7,40960,4096,16,108.0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,266,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,512,6,2,6,2,6,2,1024,2,33088,0,5,9788,1024,44894,262144.0,8192.0,1.048576e6,4096.0,4.194304e6,32768,4096,16,78.0,0,0,0,0,0,0,0,0,0,0,0,0,0
